<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Création du dataset "Charges"

**Tags:** #fec #finance #snippet #operation #charges

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** Ce notebook permet de créer le dataset pour visualiser les charges entre l'année N et N-1.

## Input

### Import libraries

In [ ]:
import naas_data_product

### Setup Variables

In [ ]:
# Inputs
input_folder_path = "/home/ftp/FEC-engine/outputs/FEC/bdd_fin_enr"

# Outputs
output_folder_path = "/home/ftp/FEC-engine/outputs/FEC/dataset_charges"

## Model

### Récupération du dernier fichier input

In [ ]:
df_input = get_last_df(input_folder_path)
print("✅ Row fetched:", len(df_input))
df_input.head(1)

### Création du dataset "CHARGES"

In [ ]:
# Creation du dataset charges
dataset_charges = df_input.copy()

# Filtre RUBRIQUE_N0 = CHARGES
dataset_charges = dataset_charges[dataset_charges["RUBRIQUE_N0"] == "CHARGES"]

# Mettre en valeur positive VALUE
dataset_charges["VALUE"] = dataset_charges["VALUE"].abs()
dataset_charges["VALUE_D"] = (dataset_charges["VALUE"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
dataset_charges["VALUE_N-1_D"] = (dataset_charges["VALUE_N-1"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
dataset_charges["VARV_D"] = (dataset_charges["VARV"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
dataset_charges.loc[dataset_charges["VARV"] > 0, "VARV_D"] = "+" + dataset_charges["VARV_D"]
dataset_charges["VARP_D"] = (dataset_charges["VARP"]).map("{:,.1%}".format)
dataset_charges.loc[dataset_charges["VARP"] > 0, "VARP_D"] = "+" + dataset_charges["VARP_D"]

# Affichage du modèle de donnée
dataset_charges

## Output

### Sauvegarde des fichiers en csv

In [ ]:
save_df(dataset_charges, output_folder_path)